# SC1015 MINI-PROJECT

OBJECTIVE: TO FIND THE BEST METHODOLOGY IN PREDICTING STOCK PRICES

SECTION 1: IMPORTING NECESSARY LIBRARIES

SECTION 2: DATA PREPROCESSING

SECTION 3: RMSE AND MAPE

SECTION 4: #1 NAIVE

SECTION 5: #2 MA-30; MA-50; MA-200

SECTION 6: #3 Exponential Smoothing

SECTION 7: #4 LSTM

SECTION 8: CONCLUSION

# SECTION 1 - IMPORTING NECESSARY LIBRARIES

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

# SECTION 2 - DATA PREPROCESSING

In [19]:
stock_cleaned_data = pd.read_csv('../data/stock_cleaned_data.csv', index_col = 0)

In [20]:
stock_cleaned_data.head()

,open,high,low,close,adjclose,volume
date,,,,,,
2023-04-18,415.58,415.72,412.7802,414.21,414.21,63559997
2023-04-17,412.37,413.96,411.0850,413.94,413.94,65496534
2023-04-14,412.81,415.09,410.0600,412.46,412.46,78161500
2023-04-13,409.18,413.84,407.9922,413.47,413.47,85814754
2023-04-12,411.87,412.17,407.4400,408.05,408.05,86420379


# SECTION 3 - RMSE AND MAPE

# SECTION 4 - #1 NAIVE

# SECTION 5 - #2 MA-30; MA-50; MA-200

# SECTION 6 - #3 EXPONENTIAL SMOOTHING

In [55]:
#without trend
length = 60
alpha = 0.3
x = np.array(stock_cleaned_data['adjclose'].iloc[:length])
f = [x[0]]
for i in range(0, len(x)):
    f.append(alpha * x[i] + (1 - alpha) * f[i])

    

In [57]:
#with trend
length = 60
alpha = 0.3
beta = 0.2
x = np.array(stock_cleaned_data['adjclose'].iloc[:length])
f = [x[0]]
t = [[1]-x[0]]
for i in range(0, len(x)):
    f.append(alpha * x[i] + (1 - alpha) * (f[i] + t[i]))
    t.append(beta * (f[i+1] - f[i]) + (1 - beta) * t[i])



# SECTION 7 - #4 LSTM

# SECTION 8 - CONCLUSION